# **Implement Seq2Seq from scratch**

---
Model: Seq2Seq Bi-GRU with Attention

Dataset: Huggingface's mt_en_vi


### Create model

In [ ]:
import torch.nn as nn
import torch
import random

In [ ]:
class RNNEncoder(nn.Module):
    def __init__(self, source_vocab_size, emb_size=100, hidden_size=512, num_layers=2, dropout_ratio=0.2, bidirectional=True):
        super().__init__()
        self.src_vocab_size = source_vocab_size
        self.hidden_size = hidden_size // 2 if bidirectional else hidden_size
        self.n_layers = num_layers
        self.n_directions = 2 if bidirectional else 1

        self.dropout = nn.Dropout(p=dropout_ratio)
        self.embedding = nn.Embedding(num_embeddings=source_vocab_size, embedding_dim=emb_size)
        self.gru = nn.GRU(input_size=emb_size, hidden_size=self.hidden_size, num_layers=num_layers,
                          bidirectional=bidirectional, dropout=dropout_ratio)
    
    def forward(self, inputs):
        # inputs: [max_input_length, bs]

        emb = self.dropout(self.embedding(inputs))
        out, hid = self.gru(emb)

        if self.n_directions == 2:
            hid = hid.view(self.n_layers, self.n_directions, -1, self.hidden_size)
            hid = torch.cat((hid[:, 0, :, :], hid[:, 1, :, :]), dim=2)
        
        return out, hid

    def load_pretrained_embedding(self):
        pass

Attention layer (Luong's version)

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size, n_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.alignment = nn.Linear(n_layers*hidden_size, hidden_size, bias=False)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, de_hid_state, en_outputs):
        # decoder hidden state: [n_layers, bs, hidden_size]
        # encoder outputs: [MAX_INPUT_LENGTH, bs, hidden_size]

        dec = de_hid_state.permute(1, 0, 2)
        bs = dec.shape[0]
        dec = dec.reshape(bs, 1, -1)
        # [bs, 1, n_layers*hid_size] --> [bs, 1, hid_size]
        dec = self.alignment(dec)

        enc = en_outputs.permute(1, 2, 0)
        # [bs, 1, hid_size] x [bs, hid_size, maxlen] -> [bs, 1, maxlen]
        score = torch.bmm(dec, enc)
        return self.softmax(score)

In [ ]:
class AttRNNDecoder(nn.Module):
    def __init__(self, target_vocab_size, emb_size=100, hidden_size=512, num_layers=2, dropout_ratio=0.2):
        super().__init__()
        self.trg_vocab_size = target_vocab_size
        self.hidden_size = hidden_size
        self.n_layers = num_layers

        self.dropout = nn.Dropout(p=dropout_ratio)
        self.embedding = nn.Embedding(num_embeddings=target_vocab_size, embedding_dim=emb_size)
        self.gru = nn.GRU(input_size=emb_size, hidden_size=hidden_size, 
                          num_layers=num_layers, dropout=dropout_ratio)
        self.attention = Attention(hidden_size, num_layers)
        self.fc1 = nn.Linear(2*hidden_size, hidden_size)
        self.gelu = nn.GELU()
        self.fc2 = nn.Linear(hidden_size, target_vocab_size)
    
    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        # input: [1, bs]
        # hidden: [n_layers, bs, hidden_size]
        # encoder_output: [MAX_INPUT_LENGTH, bs, hidden_size]

        emb = self.dropout(self.embedding(input))
        out, hid = self.gru(emb, hidden)

        att_weights = self.attention(hid, encoder_outputs)
        enc = encoder_outputs.permute(1, 0, 2)

        # [bs, 1, maxlen] x [bs, maxlen, hid_size] -> [bs, hid_size]
        context = torch.bmm(att_weights, enc).squeeze(1)
        out = out.squeeze(0)
        
        # [bs, 2*hid_size] --> [bs, hid_size]
        pred = self.fc1(torch.cat((context, out), dim=1))
        pred = self.gelu(pred)

        # [bs, hid_size] --> [bs, target_vocab_size]
        pred = self.fc2(pred)
        
        return pred, hid

    def load_pretrained_embedding(self):
        pass

In [ ]:
class AttSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.bidirectional_encoder = True if encoder.n_directions == 2 else False
        
    def forward(self, src, trg, teacher_forcing_ratio=0):
        
        # src: [max_input_length, bs]
        # trg: [max_output_length, bs]
        
        batch_size = trg.shape[1]
        max_output_len = trg.shape[0]
        trg_vocab_size = self.decoder.trg_vocab_size
        
        # tensor to store decoder outputs
        preds = torch.zeros(max_output_len, batch_size, trg_vocab_size).to(DEVICE)
    
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_outputs, hidden = self.encoder(src)
                
        # first input to the decoder is the <sos> tokens
        input = trg[0]
        
        for t in range(1, max_output_len):         
            pred, hidden = self.decoder(input, hidden, encoder_outputs)
            preds[t] = pred
            teacher_force = random.random() < teacher_forcing_ratio
            best_pred = pred.argmax(1) 
            input = trg[t] if teacher_force else best_pred
        
        return preds

### Prepare data

Load dataset

In [ ]:
!pip install datasets
from datasets import load_dataset
hf_dataset = load_dataset('mt_eng_vietnamese', 'iwslt2015-vi-en')

     |████████████████████████████████| 225kB 31.4MB/s 
     |████████████████████████████████| 245kB 38.3MB/s 
     |████████████████████████████████| 112kB 49.5MB/s 


Dataset mt_eng_vietnamese downloaded and prepared to /root/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71. Subsequent calls will reuse this data.


Preprocess data

In [ ]:
# Import hf's tokenizer
!pip install transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')

     |████████████████████████████████| 2.3MB 24.6MB/s 
     |████████████████████████████████| 901kB 49.8MB/s 
     |████████████████████████████████| 3.3MB 46.6MB/s 


In [ ]:
# Create iterator through a data set. Convert text into tensors
# Returns list of batches, each batch: {'src': [max_input_length, bs], 'trg': [max_output_length, bs]}

def make_iterator(dataset, batch_size):
    n_examples = len(dataset)
    random.shuffle(dataset)
    iterator = []
    for i in range(n_examples // batch_size):
        src_texts = []
        trg_texts = []
        for j in range(batch_size):
            src_texts.append(dataset[batch_size*i+j]['en'])
            trg_texts.append(dataset[batch_size*i+j]['vi'])
        src_tensors = tokenizer(src_texts, padding='max_length', max_length=MAX_INPUT_LENGTH, truncation=True, return_tensors='pt')['input_ids'].permute(1, 0)
        trg_tensors = tokenizer(trg_texts, padding='max_length', max_length=MAX_OUTPUT_LENGTH, truncation=True, return_tensors='pt')['input_ids'].permute(1, 0)
        new_batch = {'src': src_tensors, 'trg': trg_tensors}
        iterator.append(new_batch)
    return iterator

### Training

In [ ]:
def train(model, iterator, criterion, optimizer):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch['src'].to(DEVICE)
        trg = batch['trg'].to(DEVICE)

        optimizer.zero_grad()
        
        output = model(src, trg, TEACHER_FORCING_RATIO)

        
        # src: [max_input_length, bs]
        # trg: [max_output_length, bs]
        # output: [max_output_length, bs, trg_vocab_size]
        
        trg_vocab_size = output.shape[-1]
        
        output = output[1:].reshape(-1, trg_vocab_size)
        trg = trg[1:].reshape(-1)
        
        # trg = [(trg len - 1) * batch size]
        # output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
                
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch['src'].to(DEVICE)
            trg = batch['trg'].to(DEVICE)

            output = model(src, trg)

            # src: [max_input_length, bs]
            # trg: [max_output_length, bs]
            # output: [max_output_length, bs, trg_vocab_size]
            
            trg_vocab_size = output.shape[-1]
            
            output = output[1:].reshape(-1, trg_vocab_size)
            trg = trg[1:].reshape(-1)
            
            # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]
            
            loss = criterion(output, trg)            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# Training hyperparams
BATCH_SIZE = 64
MAX_INPUT_LENGTH = 32
MAX_OUTPUT_LENGTH = 64
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001
TEACHER_FORCING_RATIO = 0.2

# Model hyperparams
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
VOCAB_SIZE = tokenizer.vocab_size
encoder = RNNEncoder(VOCAB_SIZE).to(DEVICE)
decoder = AttRNNDecoder(VOCAB_SIZE).to(DEVICE)
seq2seq = AttSeq2Seq(encoder, decoder).to(DEVICE)

loss_function = nn.CrossEntropyLoss(ignore_index=0)     # ignore [PAD] token
optim = torch.optim.Adam(seq2seq.parameters(), lr=LEARNING_RATE)

In [ ]:
# Train model and save the best checkpoint
best_valid_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    
    # Generate train_iterator, valid_iterator
    train_iterator = make_iterator(hf_dataset['train'][:]['translation'], BATCH_SIZE)
    valid_iterator = make_iterator(hf_dataset['validation'][:]['translation'], BATCH_SIZE)

    start_time = time.time()
    
    train_loss = train(seq2seq, train_iterator, loss_function, optim)
    valid_loss = evaluate(seq2seq, valid_iterator, loss_function)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(seq2seq.state_dict(), f'att-seq2seq-{best_valid_loss:.3f}.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

Epoch: 01 | Time: 94m 28s
	Train Loss: 6.166
	 Val. Loss: 5.877
Epoch: 02 | Time: 94m 20s
	Train Loss: 5.772
	 Val. Loss: 5.718


### Inference

In [ ]:
# Load model at best checkpoint
seq2seq.load_state_dict(torch.load('att-seq2seq.pt'))

In [ ]:
# Compute loss on test set
test_iterator = make_iterator(hf_dataset['test'][:]['translation'], BATCH_SIZE)
test_loss = evaluate(seq2seq, test_iterator, loss_function)
print(f'\tTest Loss: {test_loss:.3f}')